Use Vertex AI multimodal to ask questions in a chatbot to generate
* caption
* description
* title
* rating
* category

of a product photograph

In [ ]:
%pip install --upgrade --user google-cloud-aiplatform

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

# Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:
# Define project information
from google.colab import userdata
PROJECT_ID = userdata.get('VERTEX_AI_PROJECT_ID')
LOCATION = "asia-east2"  # @param {type:"string"}

# Initialize Vertex AI
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

# Create a Gemini 1.5 Flash model

- maximum output tokens is 1024
- temperature is 1 such that the generated content is creative
- top_p is 0.95.  When the total probability of the tokens exceeds 0.95, the tokens are used for text generation

In [ ]:
from vertexai.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models
import json
from vertexai.preview.vision_models import Image

generation_config = {
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 0.95,
}

safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

model = GenerativeModel(
    "gemini-1.5-flash-001",
    system_instruction=["""You are an expert in generating fake data for an API in English."""],
    safety_settings=safety_settings,
    generation_config=generation_config
)

print(model)

# Start conversation
- Upload a image from gCloud Storage
- Create a part from the image


In [ ]:
file_path = ""  # @param {type:"string"}

# Load the image file as Image object
image = Image.load_from_file(file_path)
image.show()

imagePart = Part.from_uri(
    mime_type="image/jpeg",
    uri=file_path)

- Start a conversation
- Prompt the chatbot to generate an image caption

In [ ]:
chat = model.start_chat()

caption = chat.send_message([imagePart, """Please provide image captioning in less than 10 words in English."""]).candidates[0].content.parts[0].text
print(caption)

- Prompt the chatbot to generate the category

In [ ]:
category = chat.send_message(["""Please pick the most appropriate category from electronics, jewelery, men\'s clothing, and women\'s clothing.  Give me the category and nothing else."""]).candidates[0].content.parts[0].text
print(category)

- Prompt the chatbot to generate a title in less than 25 words
- Prompt the chatbot to generate a description based on the category and title in one paragraph. The description should be less than 300 words and in English


In [ ]:
title = chat.send_message(["""The category of the product is {category}. Please provide an English title in less than 30 words."""]).candidates[0].content.parts[0].text
print(title)

description = chat.send_message(["""Please provide the description of the product based on the above category and title in one paragraph. The description should be less than 300 words."""]).candidates[0].content.parts[0].text
print(description)

- Prompt the chatbot to generate a rating between 0 and 5 where 0 is the lowest and 5 is the highest. At most 1 decimal place is allowed.
- Print the JSON object out

In [ ]:
ratingResponse = chat.send_message(["""Please generate a rating between 0 and 5 where 0 is the lowest rating and 5 is the highest rating. At most 1 decimal place is allowed."""])
rating = ratingResponse.candidates[0].content.parts[0].text
print(rating)

object = {
    "caption": caption,
    "description": description,
    "title": title,
    "rating": rating,
    "category": category
}

print(json.dumps(object, indent=2))

# Put everything together

- Put the conversation in a function

In [ ]:
def generateData(file_path):
  generation_config = {
    "max_output_tokens": 1024,
    "temperature": 1,
    "top_p": 0.95,
  }

  safety_settings = {
      generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
      generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
  }

  model = GenerativeModel(
      "gemini-1.5-flash-001",
      system_instruction=["""You are an expert in generating fake data for an API in English."""],
      safety_settings=safety_settings,
      generation_config=generation_config
  )

  imagePart = Part.from_uri(
    mime_type="image/jpeg",
    uri=file_path)

  caption = chat.send_message([imagePart, """Please provide image captioning in less than 10 words in English."""]).candidates[0].content.parts[0].text
  category = chat.send_message(["""Please pick the most appropriate category from electronics, jewelery, men\'s clothing, and women\'s clothing.  Give me the category and nothing else."""]).candidates[0].content.parts[0].text
  title = chat.send_message(["""The category of the product is {category}. Please provide an English title in less than 30 words."""]).candidates[0].content.parts[0].text
  description = chat.send_message(["""Please provide the description of the product based on the above category and title in one paragraph. The description should be less than 300 words."""]).candidates[0].content.parts[0].text
  rating = chat.send_message(["""Please generate a rating between 0 and 5 where 0 is the lowest rating and 5 is the highest rating. At most 1 decimal place is allowed."""]).candidates[0].content.parts[0].text

  return {
      "catpion": caption,
      "description": description,
      "title": title,
      "rating": rating,
      "category": category
  }

url = "" # @param {type:"string"}
data = generateData(url)
 formatted_json = json.dumps(data, indent=2)
print(formatted_json)

In [ ]:
url = "" # @param {type:"string"}
data = generateData(url)
formatted_json = json.dumps(data, indent=2)
print(formatted_json)